<a href="https://colab.research.google.com/github/okadatsuneo/GBFS_Import/blob/main/gbfstocsv_docomo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

ドコモ・バイクシェアのstation_information.jsonファイルからOSMにインポートするためのcsvファイルを作成する。

In [1]:
#モジュールのインポート
import pandas as pd
import re
import json
from pathlib import Path

In [2]:
#ドライブのマウントはセッションが切れると解除される。ファイルも削除されるので、MyDrive下に置く。
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
# Googleドライブにあるstation_information.jsonを開く
# ファイルパスの結合はpathlibを使用
path = Path('/content/drive/MyDrive/python/')
file_name_station_info = 'd_station_information.json'
file_path = path / file_name_station_info
with open(file_path) as f:
    json_dict = json.load(f)

#'stations'の値を新たなリストlist_stationとする。
list_station = json_dict['data']['stations']

#list_stationの内容をDataFrameにする。
df = pd.DataFrame(list_station)

#'name'の値を'gbfs:name'にコピーする。
df['gbfs:name'] = df['name']

#nameの値から下記のように'ref'、'description'の要素に分離する。
# 新しい列の初期化
df['ref'] = ''
df['description'] = ''

# 'name'の各行の処理
for index, row in df.iterrows():
    # ピリオドがある場合
    if '.' in row['name']:
        parts = row['name'].split('.')
        df.at[index, 'ref'] = parts[0]
        df.at[index, 'name'] = '.'.join(parts[1:])

    # '-'がある場合
    elif '-' in row['name']:
        parts = row['name'].split('-')
        df.at[index, 'ref'] = parts[0]
        df.at[index, 'name'] = '-'.join(parts[1:])

    # '_'がある場合
    elif '_' in row['name']:
        parts = row['name'].split('_')
        df.at[index, 'ref'] = parts[0]
        df.at[index, 'name'] = '_'.join(parts[1:])

# 'ref'の各行の処理
# '【】'で囲まれた文字列があるデータが存在する場合があり、それに対してはカッコ内の文字を'description'列の値とする。
for index, row in df.iterrows():
    # '】'がある場合(文字コードで指定)
    if '\u3011' in row['ref']:
        df.at[index, 'description'], df.at[index, 'ref'] = map(str.strip, row['ref'].split('\u3011', 1))
        df.at[index, 'description'] = df.at[index, 'description'][1:]

# 列の値が数値のみの場合に数値に変換する関数
# 'ref'などで'001'のような文字を'1'に変換することを想定。数字以外の文字がある場合は変換しない
def convert_function(value):
    return pd.to_numeric(value, errors='ignore')

# 'station_id'列の各要素に対して数値に変換し、'gbfs:station_id'とする。
df['gbfs:station_id'] = df['station_id'].apply(convert_function)
#'ref'も数字に変換できる場合は変換する。
df['ref'] = df['ref'].apply(convert_function)

#出力に必要な列だけ選択する:df3
df3 = df[['lat','lon', 'ref', 'name', 'description','gbfs:name', 'gbfs:station_id', 'capacity']].copy()

#df3にOSM用の固定値を追加
df3['amenity'] = 'bicycle_rental'
df3['brand'] = 'Docomo Bike Share'
df3['brand:ja'] = 'ドコモ・バイクシェア'
df3['brand:wikidata'] = 'Q55533296'
df3['brand:website'] = 'https://docomo-cycle.jp/'
df3['gbfs:system_id'] = 'docomo-cycle'
df3['authentication:app'] = 'yes'
df3['fee'] = 'yes'

#範囲が日本全国に渡るので9つのエリアに分割してインポートできるようエリア情報を追加する。
# 緯度経度の範囲の中で要素数が同じになるように分割し(qcut)、ラベルを列の値として追加する。
df3['lat_label'] = pd.qcut(df3['lat'], 3, labels=False)
df3['lon_label'] = pd.qcut(df3['lon'], 3, labels=False)

#ファイル出力
df3.to_csv(path / 'd_stations.csv')
df3.to_excel(path / 'd_stations.xlsx')
print("処理が完了しました。")


処理が完了しました。
